In [28]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = X*1
    for i = 1:size(X)[1]
        L[i]= intersect(X[i],Y[i])
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=0
    max_index=0
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
    end
    
    if diam(X[max_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end

function border(K,I)
    c=0.9
    K_width=diam.(K)
    I_width=diam.(I)
    max=0
    for i=1:size(K)[1]
        if (K_width[i]/I_width[i])>max
            max=K_width[i]/I_width[i]
        end
    end
    if max<=0.9
        return true
    else
        return false
    end
end

function remove_duplication(S)
    tmp_index =[]
    tmp_array=[]
    for i=1:size(S)[1]-1
        for j = i+1:size(S)[1]
            if !orig_is_empty(S[i],S[j])
                
                if (i in tmp_index) || (j in tmp_index)
                    push!(tmp_index,i)
                    push!(tmp_index,j)
                    continue
                end
                push!(tmp_index,i)
                push!(tmp_index,j)
                I = orig_intersect(S[i],S[j])
                push!(tmp_array,I)
            end
        end
    end
    for i=1:size(S)[1]
        if !(i in tmp_index)
            push!(tmp_array,S[i])
        end
    end
    
    return tmp_array
end


function allsol(F,X)
    cnt=0
    L = [X] #調査する区間のリスト
    S = Array[] #見つかった解のリスト
    
    while (size(L)[1] != 0)
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(X)) ) 
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(c)+j*(X-c)) )
            continue
        end
        
        R = []
        try
            R = inv(ForwardDiff.jacobian(F,c))
        catch e
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
                continue
            catch e
                println(e)
            end
            println(":逆行列の演算に失敗")
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        
        if (orig_is_empty(K,X))
            continue #解なし
        end
        
        if (orig_is_subset(K,X))　#解あり
            push!(S, K) #解のリストに追加
            continue
        end
        
        # 境界に乗ってた時
        if border(K,X)
            push!(L,K)
            continue
        end
        
        try
            X1,X2 = orig_devide(orig_intersect(K,X))
            #K and X を2分割
            push!(L, X1)
            push!(L, X2)
            continue
        catch e
            println(e)
        end
    end
    
    S = remove_duplication(S)
    
    #区間を狭める
    for i =1:size(S)[1]
        si=S[i]
        while(maximum(radius,si) >= 10^-5)
            j=0
            try
                j = ForwardDiff.jacobian(F,si)
            catch
                break
            end
            c = mid.(si)
            R=0
            try 
                R = inv(ForwardDiff.jacobian(F,c)) 
            catch
                break
            end
            M = Matrix{Float64}(I,size(R))-R*j
            k = c -R*F(c)+M*(si-c)
            if orig_is_subset(k,si)
                si=k
            else
                break
            end
        end
        S[i]=si
    end
    
    for s=1:size(S)[1] 
        print("解");print(s);print(": ");
        for i in S[s]
            print(i)
            print(" , ")
        end
        println("")
    end
    
    return S
end

# 重解　＝＞区間幅に制限をつける

allsol (generic function with 1 method)

In [29]:
X = [(-1000. ..1000.),(-1000. .. 1000.)]

# f((x1, x2)) = x1^2 + x2^2 - 1
# g((x1, x2)) = x1^2 - x2^4

# f((x1,x2)) = x1 + 2*x2 +1
# g((x1,x2)) = x1^2 + 2*(x2^2) -1

f((x1, x2)) = 2*x1^3 + 2*x1*x2 - 22*x1+ x2^2 + 10 + pi
g((x1, x2)) = x1^2 + 2*x1*sin(x2) + 2*x2^3 - 14*x2 + 9.1

F((x,y))=[f((x,y));g((x,y))]
answer=allsol(F,X)
@show answer


解1: [-4.14462, -4.14461] , [-3.28814, -3.28813] , 
解2: [3.2574, 3.25741] , [-3.18413, -3.18412] , 
解3: [0.820019, 0.820025] , [-2.93392, -2.93389] , 
解4: [1.08436, 1.08437] , [1.97179, 1.9718] , 
解5: [-3.4131, -3.41309] , [1.69858, 1.69859] , 
解6: [0.720303, 0.720304] , [0.853378, 0.853379] , 
解7: [-3.43452, -3.43451] , [1.40441, 1.40442] , 
answer = Any[Interval{Float64}[[-4.14462, -4.14461], [-3.28814, -3.28813]], Interval{Float64}[[3.2574, 3.25741], [-3.18413, -3.18412]], Interval{Float64}[[0.820019, 0.820025], [-2.93392, -2.93389]], Interval{Float64}[[1.08436, 1.08437], [1.97179, 1.9718]], Interval{Float64}[[-3.4131, -3.41309], [1.69858, 1.69859]], Interval{Float64}[[0.720303, 0.720304], [0.853378, 0.853379]], Interval{Float64}[[-3.43452, -3.43451], [1.40441, 1.40442]]]


7-element Vector{Any}:
 Interval{Float64}[[-4.14462, -4.14461], [-3.28814, -3.28813]]
 Interval{Float64}[[3.2574, 3.25741], [-3.18413, -3.18412]]
 Interval{Float64}[[0.820019, 0.820025], [-2.93392, -2.93389]]
 Interval{Float64}[[1.08436, 1.08437], [1.97179, 1.9718]]
 Interval{Float64}[[-3.4131, -3.41309], [1.69858, 1.69859]]
 Interval{Float64}[[0.720303, 0.720304], [0.853378, 0.853379]]
 Interval{Float64}[[-3.43452, -3.43451], [1.40441, 1.40442]]